<a href="https://colab.research.google.com/github/mp11-rohan/Recommendation-Systems/blob/main/Sistemas_Recomendadores_Experiencia_Matem%C3%A1tica_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357231 sha256=cb0be8a0db11055aaea9335092996a85b49481d0ba3166a4ed6471913812153e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load the MovieLens dataset
!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

--2024-07-02 20:50:24--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.53MB/s    in 0.2s    

2024-07-02 20:50:24 (4.53 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [4]:
# Read the dataset into a pandas dataframe
ratings = pd.read_csv('/content/ratings.csv')
#movies
clothes = pd.read_csv('/content/clothes.csv')

# Prepare the data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'clothesId', 'ratings']], reader)

In [5]:
# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Calculate RMSE
print("RMSE: ", rmse(predictions))

RMSE: 1.3274
RMSE:  1.3274236600589846


In [6]:
# Function to get a list of clothes ids that the user has not rated
def get_unrated_clothes(user_id, ratings_df, clothes_df):
    rated_clothes = ratings_df[ratings_df['userId'] == user_id]['clothesId']
    all_clothes = set(clothes_df['clothesId'])
    rated_clothes_set = set(rated_clothes)
    unrated_clothes = list(all_clothes - rated_clothes_set)
    return unrated_clothes

In [7]:
# Function to recommend clothes for a user
def recommend_clothes(user_id, ratings_df, clothes_df, algo, top_n=10):
    unrated_clothes = get_unrated_clothes(user_id, ratings_df, clothes_df)
    predictions = [algo.predict(user_id, clothes_id) for clothes_id in unrated_clothes]
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    recommended_clothes_ids = [pred.iid for pred in top_predictions]
    recommended_clothes = clothes_df[clothes_df['clothesId'].isin(recommended_clothes_ids)]
    return recommended_clothes

In [27]:
# Get recommendations for a specific user
user_id = 1
recommended_clothes = recommend_clothes(user_id, ratings, clothes, algo, top_n=10)
print(f"Top 10 clothes recommendations for user {user_id}:")
print(recommended_clothes[['clothesName', 'genres']])

Top 10 clothes recommendations for user 1:
           clothesName  genres
26   Button-down shirt     Men
34        Biker jacket  Unisex
38       Bodycon dress   Women
42      Leather jacket  Unisex
72      Peasant blouse   Women
80        Track jacket  Unisex
116          Tracksuit     Men
137      Checked shirt     Men
159     Slim fit shirt     Men
183              Parka  Unisex
